In [85]:
import requests
import numpy as np
import math
import os
import shutil

In [86]:
base_url = "https://api.mangadex.org"
title = "Karakai Jouzu no Takagi-san"

r = requests.get(
    f"{base_url}/manga",
    params={"title": title}
)

for i in range(len(r.json()["data"])):
    print(f"[{i}]: {r.json()['data'][i]['attributes']['title']}")


[0]: {'en': 'Karakai Jouzu no Takagi-san'}
[1]: {'en': 'Karakai Jouzu no (Moto) Takagi-san'}
[2]: {'en': 'Family Complete +'}
[3]: {'ja': 'Karakai Jouzu no Takagi-san Colored Edition'}
[4]: {'en': 'Karakai Jouzu no Takagi-san - Pokemon Trainer Takagi-san (Doujinshi)'}
[5]: {'en': 'Karakai Jouzu no Takagi-san - Karakai Jouzu no Takagi-san Fan Comic (Doujinshi)'}
[6]: {'en': 'Kirakira PreCure a la Mode! - Teasing Master Yukari-san'}


In [87]:
titleid = r.json()["data"][0]["id"]
print(titleid)

6e445564-d9a8-4862-bff1-f4d6be6dba2c


In [95]:
total = 0
i = 0
lschap = [] 
while (i == 0 or i * 100 <= total):
    r = requests.get(
        f"{base_url}/manga/{titleid}/feed",
        params={"offset":i * 100}
    )

    r_json = r.json()
    for c in r_json["data"]:
        if (c["attributes"]["translatedLanguage"] == "en"):
            lschap.append(c)

    i += 1
    total = r_json["total"]

sc = sorted(lschap, key=lambda d: float(d["attributes"]["chapter"]))

scid = []
for i in sc:
    scid.append(i["id"])

In [97]:
print(len(sc))

216


In [39]:
# bulk = np.arange(0, 5)
bulk = [0]
prev = -1
chapnums = []
for chap in bulk:
    chapnum = sc[chap]["attributes"]["chapter"]
    if (prev == chapnum):
        continue
    chapnums.append(chapnum)

    r = requests.get(
        f"{base_url}/at-home/server/{scid[chap]}",
    )

    r_json = r.json()

    url = []
    pagesid = r_json["chapter"]["data"]
    base = r_json["baseUrl"]
    hash = r_json["chapter"]["hash"]

    if (not "mangadex.org" in base):
        print("uh oh")

    for id in pagesid:
        url.append(f"{base}/data/{hash}/{id}")


    for i in range(len(url)):
        page = requests.get(url[i]).content
        with open(f'{chapnum}-{i+1}.jpg', 'wb') as handler:
            handler.write(page)
    prev = chapnum

In [96]:
for i in chapnums:
    os.makedirs(f"./{title}/chapter {i}")

ls = os.listdir()
ls.sort()

jpg = []
ch = []
prev = chapnums[0]
for file in ls:
    ext = file.split('.')[-1]
    if (ext != "jpg"):
        continue
    curr = file.split("-")[0]
    if (curr != prev):
        jpg.append(ch)
        ch = []
    ch.append(file[0:-4])
    prev = curr

jpg.append(ch)

for i in range(len(jpg)):
    currchap = jpg[i]
    simg = sorted(currchap, key=lambda d: int(d.split("-")[-1]))
    jpg[i] = simg

for i in jpg:
    for j in i:
        file = f"./{j}.jpg"
        c = j.split("-")[0]
        shutil.move(file, f"./{title}/chapter {c}")

In [33]:
print(chap)

185
